# Introduction to WaveNet
## Jonathan Balaban

WaveNet is a powerful new predictive technique that uses multiple Deep Learning (DL) strategies from Computer Vision (CV) and Audio Signal Processing models and applies them to longitudinal (time-series) data. It was created by researchers at London-based artificial intelligence firm [DeepMind](https://deepmind.com), and currently powers [Google Assistant voices](https://assistant.google.com).

We'll explore WaveNet and how it works, but first dive into data prep, current high-performance models (as a baseline, Facebook Prophet), and then compare results!

>run this full line in bash, as one line:

`conda create -n wavenet python=3.7 pandas numpy seaborn matplotlib jupyter keras`

>then install Facebook Prophet:

`source activate wavenet; conda install -c conda-forge fbprophet`

# Data Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:
# load Apple stock
aapl_raw = pd.read_csv('./AAPL-2000-2019.csv')

In [ ]:
aapl_raw.info()

# we need to set date as datetime object and index

In [ ]:
# set index
aapl_raw.index = pd.to_datetime(aapl_raw.Date)

In [ ]:
# check
aapl_raw.head()

# looks good!

In [ ]:
# rename Adj Close to AC, get rid of pesky spacing
aapl_raw['AC'] = aapl_raw['Adj Close']

>Based on [this article](https://finance.zacks.com/adjusted-closing-price-vs-closing-price-9991.html) and others, let's use Adj Close as our series.

>[MACD is often used by investors as a buy/sell indicator](https://www.investopedia.com/terms/m/macd.asp), so it should help track patterns and trend changes!

In [ ]:
# good, drop Date and everything but AC and Volume
aapl = aapl_raw.drop(columns=['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close'])
aapl.head()

In [ ]:
aapl.describe().round(2)

In [ ]:
sns.pairplot(aapl);

We can note a few things from the EDA above:
- No NULLs
- Inverse exponential relationship between AC and Volume?
- Right skew in price

There's an important consideration also: **stock splits!**

Apple's stock has split four times since the company went public. The stock split on a 7-for-1 basis on June 9, 2014 and split on a 2-for-1 basis on February 28, 2005, June 21, 2000, and June 16, 1987.

Let's filter to the 2014 and 2005 weeks and see what happened:

# Statistics

In [ ]:
# plot price over time
aapl.AC.plot();

In [ ]:
# Optional: run ADF to determine unit root
import statsmodels.tsa.stattools as ts
cadf = ts.adfuller(aapl_raw.AC)

print('Augmented Dickey Fuller:')
print('Test Statistic =',cadf[0])
print('p-value =',cadf[1])
print('Critical Values =',cadf[4])

# FB Prophet

# WaveNet

### Dilated Causal Convolution Layers
The model architecture we will build is a variant of the [WaveNet model](https://deepmind.com/blog/wavenet-generative-model-raw-audio/), but optimized for sequence data.

The **dilated causal convolution layer** is the default building block; Joe Eddy has a fantastic breakdown of [how it works here](https://github.com/JEddy92/TimeSeries_Seq2Seq/blob/master/notebooks/TS_Seq2Seq_Conv_Intro.ipynb) and [in his blog here](https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/). In a nutshell, these layers (borrowed and modified from CNNs) are designed to learn detailed patterns from recent stock price movements, but also big-picture trends and cycles from months and years ago.

![dilated convolutions](./images/WaveNet_dilatedconv.png)

In [ ]:
# looking back a month and year
plt.plot(aapl.AC.iloc[-30:], 'r', linewidth=8, markersize=12);
plt.plot(aapl.AC.iloc[-365:]);

# what intel can we gather from each timeframe?

### More building blocks:

We need to add a few other techniques:

![blocks](./images/WaveNet_residblock.png)

### **Gated Activations**

In the boxed portion of the architecture diagram above, note the dilated convolution splits into two branches that recombine via element-wise multiplication. This is a **gated activation unit**:
- **tanh:** activation branch is a learned (model optimized) filter
- **sigmoid:** activation branch is a learned gate that controls flow from the *tanh* filter (just like LSTM gates)

### Skip Connections

In fully-connected NNs, a neuron takes inputs from all neurons in the previous layer: early layers establish later ones via a hierarchy of intermediate computations. This allows NNs to build complex interactions of raw inputs/signals.

But... what if raw inputs are directly useful for prediction, and we want them to directly influence the output? **Skip connections** allow outputs of any layer to bypass multiple future layers and skip influence dilution! Keras allows us to store the tensor output of each convolutional block - in addition to passing it through further layers - with `skips.append()`. Note how for each block in the stack above, the output from the gated activations joins the set of skip connections. How much or how far you do this is a hyper-parameter that depends on your data and model structure!

### Residual Connections

![CNN_skips](./images/CNN_skips.png)

**Residual connections** are similar to skip connections: think of them as consistenly-available short layer skips! We'll use a one-layer skip for our model, but it's also a hyper-parameter. Why they help is mysterious, but it's most likely due to helping with the [Vanishing or Exploding Gradient](https://medium.com/learn-love-ai/the-curious-case-of-the-vanishing-exploding-gradient-bf58ec6822eb) obstacle in backpropagation. This becomes more important with larger models, but I'll show you the implementation in a smaller setting for educational purposes.

> Note: the diagram's *1x1 convolutions* are basically time-distributed fully connected layers; this allows us to use CNN language for our architecting!

![XKCD](./images/XKCD.png)

## Data Setup

First, Keras needs us to convert our series into `numpy` arrays. As we prep our data, we should have a partition strategy to validate and test our model's results. [Artur Suilin](https://github.com/Arturus/kaggle-web-traffic/blob/master/how_it_works.md) created a visualization of the **walk-forward validation** strategy that serves as a best practice:

![walk-forward validation](./images/ArturSuilin_validation.png)

Let's use this approach as we parse, learn, and validate our data!

In [ ]:
# transpose to fit structure below
aapl = aapl[['Volume', 'AC']]
aapl = aapl.T
aapl.head()

In [ ]:
from datetime import timedelta

data_start_date = aapl.columns[1]
data_end_date = aapl.columns[-1]

pred_steps = 14
pred_length=timedelta(pred_steps)

first_day = pd.to_datetime(data_start_date) 
last_day = pd.to_datetime(data_end_date)

val_pred_start = last_day - pred_length + timedelta(1)
val_pred_end = last_day

train_pred_start = val_pred_start - pred_length
train_pred_end = val_pred_start - timedelta(days=1) 

In [ ]:
enc_length = train_pred_start - first_day

train_enc_start = first_day
train_enc_end = train_enc_start + enc_length - timedelta(1)

val_enc_start = train_enc_start + pred_length
val_enc_end = val_enc_start + enc_length - timedelta(1) 

In [ ]:
print('Train encoding:', train_enc_start, '-', train_enc_end)
print('Train prediction:', train_pred_start, '-', train_pred_end, '\n')
print('Val encoding:', val_enc_start, '-', val_enc_end)
print('Val prediction:', val_pred_start, '-', val_pred_end)

print('\nEncoding interval:', enc_length.days)
print('Prediction interval:', pred_length.days)

In [ ]:
date_to_index = pd.Series(index=pd.Index([pd.to_datetime(c) for c in aapl.columns[1:]]),
                          data=[i for i in range(len(aapl.columns[1:]))])

series_array = aapl[aapl.columns[1:]].values

def get_time_block_series(series_array, date_to_index, start_date, end_date):
    
    inds = date_to_index[start_date:end_date]
    return series_array[:,inds]

def transform_series_encode(series_array):
    
    series_array = np.log1p(np.nan_to_num(series_array)) # filling NaN with 0
    series_mean = series_array.mean(axis=1).reshape(-1,1) 
    series_array = series_array - series_mean
    series_array = series_array.reshape((series_array.shape[0],series_array.shape[1], 1))
    
    return series_array, series_mean

def transform_series_decode(series_array, encode_series_mean):
    
    series_array = np.log1p(np.nan_to_num(series_array)) # filling NaN with 0
    series_array = series_array - encode_series_mean
    series_array = series_array.reshape((series_array.shape[0],series_array.shape[1], 1))
    
    return series_array

# WaveNet Modeling

## Architecture Overview

Let's build our model like so:

* 16 dilated causal convolutional blocks
    * Pre and postprocessing (time distributed) fully connected layers (i.e. convolutions with filter width 1):
        * 16 output units
    * 32 filters of width 2 per block
    * Exponentially increasing dilation rate with a reset (1, 2, 4, 8,... 2^6), two sets
    * Gated activations
    * Residual and skip connections
* 2 fully connected layers to combine skip outputs to final output

In [ ]:
from keras.models import Model
from keras.layers import Input, Conv1D, Dense, Activation, Dropout, Lambda, Multiply, Add, Concatenate
from keras.optimizers import Adam

In [ ]:
# convolutional operation parameters
n_filters = 16 
filter_width = 2
dilation_rates = [2**i for i in range(5)] * 2 

# define an input history series and pass it through a stack of dilated causal convolution blocks. 
history_seq = Input(shape=(None, 1))
x = history_seq

skips = []
for dilation_rate in dilation_rates:
    
    # preprocessing - equivalent to time-distributed dense
    x = Conv1D(16, 1, padding='same', activation='relu')(x) 
    
    # filter convolution
    x_f = Conv1D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='causal',
                 dilation_rate=dilation_rate)(x)
    
    # gating convolution
    x_g = Conv1D(filters=n_filters,
                 kernel_size=filter_width, 
                 padding='causal',
                 dilation_rate=dilation_rate)(x)
    
    # multiply filter and gating branches
    z = Multiply()([Activation('tanh')(x_f),
                    Activation('sigmoid')(x_g)])
    
    # postprocessing - equivalent to time-distributed dense
    z = Conv1D(16, 1, padding='same', activation='relu')(z)
    
    # residual connection
    x = Add()([x, z])    
    
    # collect skip connections
    skips.append(z)

# add all skip connection outputs 
out = Activation('relu')(Add()(skips))

# final time-distributed dense layers 
out = Conv1D(128, 1, padding='same')(out)
out = Activation('relu')(out)
out = Dropout(.2)(out)
out = Conv1D(1, 1, padding='same')(out)

# extract the last 60 time steps as the training target
def slice(x, seq_length):
    return x[:,-seq_length:,:]

pred_seq_train = Lambda(slice, arguments={'seq_length':10})(out)

model = Model(history_seq, pred_seq_train)
model.compile(Adam(), loss='mean_absolute_error')

In [ ]:
# print our model architecture
#model.summary()

In [ ]:
first_n_samples = 2

# sample of series from train_enc_start to train_enc_end  
encoder_input_data = get_time_block_series(series_array, date_to_index, 
                                           train_enc_start, train_enc_end)[:first_n_samples]
encoder_input_data, encode_series_mean = transform_series_encode(encoder_input_data)

# sample of series from train_pred_start to train_pred_end 
decoder_target_data = get_time_block_series(series_array, date_to_index, 
                                            train_pred_start, train_pred_end)[:first_n_samples]
decoder_target_data = transform_series_decode(decoder_target_data, encode_series_mean)

# we append a lagged history of the target series to the input data, 
# so that we can train with teacher forcing
lagged_target_history = decoder_target_data[:,:-1,:1]

encoder_input_data = np.concatenate([encoder_input_data, lagged_target_history], axis=1)

In [ ]:
model.compile(Adam(), loss='mean_absolute_error')

history = model.fit(encoder_input_data, decoder_target_data,
                    batch_size=2**6,
                    epochs=100,
                    validation_split=0.2)

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.xlabel('Epoch')
plt.ylabel('Mean Absolute Error Loss')
plt.title('Loss Over Time')
plt.legend(['Train','Validation']);

# Additional Resources

Many thanks to my amazing colleague [Joe Eddy](https://www.linkedin.com/in/joseph-eddy-178425129/) for his informative blog posts on WaveNet:
- [Time Series Forecasting with Convolutional Neural Networks - a Look at WaveNet](https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv/)
- [Time Series Forecasting with Convolutional Neural Networks - Further Exploration of WaveNet](https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_conv2/)
- Optional context: [Forecasting with Neural Networks - An Introduction to Sequence-to-Sequence Modeling Of Time Series](https://jeddy92.github.io/JEddy92.github.io/ts_seq2seq_intro/)


## WaveNet Official
- [Blog on audio applications](https://deepmind.com/blog/wavenet-generative-model-raw-audio/)
- [Voice showcase](https://cloud.google.com/text-to-speech/docs/wavenet)
- [Paper](https://arxiv.org/abs/1609.03499)
- [Wiki](https://en.wikipedia.org/wiki/WaveNet)